In [10]:
import mesa
import os

mesa_dir = os.path.dirname(mesa.__file__)
print(mesa.__version__)
print(mesa.__file__)
print("Mesa directory:", mesa_dir)
print("Contents:", os.listdir(mesa_dir))
import random
random.seed(0)
import warnings
warnings.filterwarnings("ignore")
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

sys.path.append("C:/Users/met48/Desktop/TS-Clustering/mesa-examples-main/mesa-examples-main/examples/bank_reserves/") 
from bank_reserves import random_walk as bankReservesRandomWalk
from bank_reserves import agents as bankReservesAgent
from bank_reserves import model as bankReservesModel

sys.path.append("C:/Users/met48/Desktop/TS-Clustering/mesa-examples-main/mesa-examples-main/examples/epstein_civil_violence/") 
from epstein_civil_violence import __init__ as epsteinInit
from epstein_civil_violence import agent as epsteinCVAgent
from epstein_civil_violence import model as epsteinCVModel

import time
from sklearn.preprocessing import MinMaxScaler

3.2.1.dev
C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\mesa\__init__.py
Mesa directory: C:\Users\met48\AppData\Local\anaconda3\Lib\site-packages\mesa
Contents: ['agent.py', 'batchrunner.py', 'datacollection.py', 'discrete_space', 'examples', 'experimental', 'mesa_logging.py', 'model.py', 'space.py', 'visualization', '__init__.py', '__pycache__']


In [2]:
def generate_epstein_samples(inputs):  
    all_outputs = []

    for i in range(len(inputs)):
        if i % 100 == 0:
            print(i)
        row = inputs.iloc[i]
        cit_dens = row[0]
        cop_dens = row[1]
        leg = row[2]
        
        # Run the Epstein model
        bankRes = epsteinCVModel.EpsteinCivilViolence(
            citizen_density=cit_dens,
            cop_density=cop_dens,
            citizen_vision=5,
            cop_vision=5,
            legitimacy=leg,
            max_jail_term=30,
            active_threshold=0.1,
            arrest_prob_constant=2.3,
            movement=True,
            max_iters=250
        )
        bankRes.run_model()

        # Collect the time series of "Active" agents
        results = bankRes.datacollector.get_model_vars_dataframe()
        active_series = results['Active'].to_list()

        # Add to list of all outputs
        all_outputs.append(active_series)

    # Convert to DataFrame: each row = one run, columns = time steps
    outputs_df = pd.DataFrame(all_outputs)
    return outputs_df


In [8]:
def generate_br_samples(inputs):
    
    all_outputs = []

    for i in range(len(inputs)):
        if i % 100 == 0:
            print(i)
        row = inputs.iloc[i]
        reserve_perc = row[0]
        
        # Run the Epstein model
        bankRes = bankReservesModel.BankReserves(init_people=500, rich_threshold=10, reserve_percent=reserve_perc)
        bankRes.run_model()

        # Collect the time series of "Active" agents        
        results = bankRes.datacollector.get_model_vars_dataframe()
        active_series = results['Poor'].to_list()

        # Add to list of all outputs
        all_outputs.append(active_series)

    # Convert to DataFrame: each row = one run, columns = time steps
    outputs_df = pd.DataFrame(all_outputs)
    return outputs_df

In [4]:
def evaluate_model_gan(real_samples, generated_samples):

    # convert list to numpy arrays
    predicted_values = generated_samples.to_numpy()  # [N, 255, 1]
    true_values = real_samples.to_numpy()            # [N, 255, 1]

    # flatten for metrics
    y_pred_flat = predicted_values.flatten()
    y_true_flat = true_values.flatten()
    
    # compute evaluation metrics
    mse = mean_squared_error(y_true_flat, y_pred_flat)
    mae = mean_absolute_error(y_true_flat, y_pred_flat)
    r2 = r2_score(y_true_flat, y_pred_flat)

    print(f"Validation MSE: {mse:.6f}")
    print(f"Validation MAE: {mae:.6f}")
    print(f"Validation R² Score: {r2:.6f}")

    return mse, mae, r2

# Bank Reserves

In [11]:
valid_inputs_br = pd.read_csv("valid_inputs_br.csv")
valid_outputs_br = pd.read_csv("valid_outputs_br.csv")

In [12]:
df_br = generate_br_samples(valid_inputs_br)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


In [13]:
df_br.to_csv("validation_set_dup_br.csv", index=False)

In [14]:
scaler = MinMaxScaler()

valid_outputs_br_std = scaler.fit_transform(valid_outputs_br)
valid_outputs_br_std = pd.DataFrame(valid_outputs_br_std)

df_br_std = scaler.fit_transform(df_br)
df_br_std = pd.DataFrame(df_br_std)

evaluate_model_gan(valid_outputs_br_std, df_br_std)

Validation MSE: 0.006567
Validation MAE: 0.052487
Validation R² Score: 0.934792


(0.006567412579512853, 0.0524874867984366, 0.934791711922361)

# Epstein

In [15]:
valid_inputs_epstein = pd.read_csv("valid_inputs_epstein.csv")
valid_outputs_epstein = pd.read_csv("valid_outputs_epstein.csv")

In [ ]:
df_epstein = generate_epstein_samples(valid_inputs_epstein)

0


In [ ]:
df_epstein.to_csv("validation_set_dup_ecv.csv", index=False)

In [ ]:
scaler = MinMaxScaler()

valid_outputs_ecv_std = scaler.fit_transform(valid_outputs_epstein)
valid_outputs_ecv_std = pd.DataFrame(valid_outputs_ecv_std)

df_ecv_std = scaler.fit_transform(df_epstein)
df_ecv_std = pd.DataFrame(df_ecv_std)

evaluate_model_gan(valid_outputs_br_std, df_br_std)